In [20]:
from langchain.llms import Ollama
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
import os
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Initialize model mistral
model = Ollama(model="mistral_copy")

In [3]:
pdf_path = "/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf"

In [4]:
loader = PyMuPDFLoader(pdf_path)
doc = loader.load()

In [5]:
# Définir les séparateurs pour le découpage du texte
separators = [
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
]

In [6]:
# Initialiser le découpeur de texte avec les paramètres spécifiés
text_splitter = RecursiveCharacterTextSplitter(
    separators=separators,
    chunk_size=300, # Taille de chaque chunk en caractères
    chunk_overlap=50, # Chevauchement entre les chunks consécutifs (réduit pour moins de chunks)
    length_function=len, # Fonction pour calculer la longueur du texte
    add_start_index=True, # Ajouter l'index de début à chaque chunk
)

In [7]:

chunks = text_splitter.split_documents(doc)
print(f"Split {len(doc)} documents into {len(chunks)} chunks.")

Split 2 documents into 13 chunks.


In [8]:
# Afficher un exemple de contenu de page et de métadonnées pour un chunk
page = chunks[0]
print(page.page_content)
print(page.metadata)

Kalaban Coura ACI, Bamako, Rue 418 
Tel : +223 62 09 21 57 
Email : mohameddiallo728@gmail.com 
MOHAMED DIALLO 
Maîtrisant des outils tels que MySQL, PostgreSQL, je suis capable de façonner et d'optimiser des bases de données
{'source': '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', 'file_path': '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', 'page': 0, 'total_pages': 2, 'format': 'PDF 1.7', 'title': '', 'author': 'Mohamed DIALLO', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2019', 'producer': 'Microsoft® Word 2019', 'creationDate': "D:20240216080534+00'00'", 'modDate': "D:20240216080534+00'00'", 'trapped': '', 'start_index': 0}


In [9]:
# Chemin vers le répertoire pour enregistrer la base de données Chroma
CHROMA_PATH = "embdb"
EMB_CACHE_PATH = "emb_cache.pkl"

In [10]:
# Récupérer la fonction d'embeddings à partir des ressources du code env
emb_model = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=emb_model)

/home/mohamed/envpy/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/home/mohamed/envpy/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-07-25 16:48:14.322712: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the e

In [16]:
# Charger les embeddings depuis le cache ou les générer si le cache n'existe pas
if os.path.exists(EMB_CACHE_PATH):
    with open(EMB_CACHE_PATH, 'rb') as f:
        cached_embeddings = pickle.load(f)
else:
    # Générer les embeddings pour chaque chunk
    cached_embeddings = embeddings.embed_documents([chunk.page_content for chunk in chunks])
    with open(EMB_CACHE_PATH, 'wb') as f:
        pickle.dump(cached_embeddings, f)

In [26]:
# Créer une nouvelle base de données Chroma et ajouter les documents
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
db.add_documents(documents=chunks)

['3d27d286-20a3-4c72-a32f-c7061c9ced4f',
 '35411716-7281-4414-9fba-1e749f94ca25',
 '76ec560e-aae2-4403-8322-95ebff47076d',
 '61652b31-285e-4cb1-b743-6d75e3ecc685',
 '5bf65004-e650-4e69-b23f-4636961b32b6',
 '81e9a912-d5bf-47e5-bf6a-c8daa6dc8d7c',
 'cedf7036-58de-46cb-8ca0-a6a944e4add8',
 '7229ce9b-9eb2-45ea-872f-1e9e2c2d3bde',
 'f6183037-2962-460b-baa3-dfc8f80d6a31',
 '5dc78727-39bf-4177-85e1-efc51d27bd1a',
 '38849445-453b-47f3-b686-0ea4b7069a63',
 '1adce40c-88bb-4a63-8c7e-236a99035115',
 '3e8622d8-aa45-4e9b-986f-d5eb7e1a1c72']

In [27]:
# Persister la base de données sur le disque
db.persist()
print(f"Enregistrement de {len(chunks)} chunks dans {CHROMA_PATH}.")

Enregistrement de 13 chunks dans embdb.


/home/mohamed/envpy/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [29]:
query_text = "Quelles competences Mohamed possède t'il ?"

In [30]:
PROMPT_TEMPLATE = """
Tu t'appelles Okka.
Tu es un assistant en intelligence artificielle conçu pour aider les utilisateurs en récupérant des informations pertinentes et en générant des réponses basées sur ces informations. 
Ton objectif est de fournir des réponses claires et utiles.

L'utilisateur a posé la question suivante : "{question}"
Recherche des documents pertinents dans la base de connaissances et utilise ces informations pour répondre à la question.

Réponds à la question en utilisant uniquement le contexte suivant :
{context}
- -
Réponds à la question ci-dessous en te basant uniquement sur le contexte fourni, dans la même langue que la question :
Question : {question}
"""

In [48]:
 # Prepare the database
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embeddings)
  
# Récupérer le contexte à partir de la DB en utilisant la recherche par similarité
results = db.similarity_search_with_relevance_scores(query_text, k=5)

/home/mohamed/envpy/lib/python3.12/site-packages/langchain_core/vectorstores/base.py:784: UserWarning: Relevance scores must be between 0 and 1, got [(Document(metadata={'author': 'Mohamed DIALLO', 'creationDate': "D:20240216080534+00'00'", 'creator': 'Microsoft® Word 2019', 'file_path': '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20240216080534+00'00'", 'page': 0, 'producer': 'Microsoft® Word 2019', 'source': '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', 'start_index': 227, 'subject': '', 'title': '', 'total_pages': 2, 'trapped': ''}, page_content="pour répondre aux exigences spécifiques de chaque projet. Mon expertise s'étend également à une gamme diversifiée \nde langages de programmation, de PHP à Python, en passant par Java et bien d'autres, me permettant ainsi d'aborder \nles défis de manière polyvalente. \nEXPÉRIENCES"), 0.023655544853531607), (Document(metadata={'author': 'Mohamed DIALLO', 'creationDate'

In [49]:
if len(results) == 0 or results[0][1] < 0.7:
    print(f"Impossible de trouver des résultats correspondants.")
else:
    # Combiner le contexte des documents correspondants
    context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])

    # Créer un modèle de prompt en utilisant le contexte et le texte de la requête
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)

    # Générer le texte de réponse en fonction du prompt
    response_text = model.predict(prompt)

    print(response_text)

    # Obtenir les sources des documents correspondants
    sources = [doc.metadata.get("source", None) for doc, _score in results]

    # Formater et retourner la réponse incluant le texte généré et les sources
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)

Impossible de trouver des résultats correspondants.


In [43]:
# Combiner le contexte des documents correspondants
context_text = "\n\n - -\n\n".join([doc.page_content for doc, _score in results])

 
# Créer un modèle de prompt en utilisant le contexte et le texte de la requête
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context=context_text, question=query_text)

In [45]:
# Générer le texte de réponse en fonction du prompt
response_text = model.invoke(prompt)

In [46]:
# Obtenir les sources des documents correspondants
sources = [doc.metadata.get("source", None) for doc, _score in results]

In [47]:
# Formater et retourner la réponse incluant le texte généré et les sources
formatted_response = f"Response: {response_text}\nSources: {sources}"

print(formatted_response)

Response:  Mohamed possède des compétences en base de données, notamment MySQL et PostgreSQL. Il est également expérimenté dans une gamme variée de langages de programmation allant de PHP à Python en passant par Java et React.js. Son profil technique comprend également TypeScript, SpringBoot et Flask.
Sources: ['/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf', '/home/mohamed/Documents/Mohamed/Mohamed_DIALLO_CV.pdf']
